In [4]:
import json
import csv
import os
import select
from pprint import pprint as pr
import sqlalchemy as sa
import pandas as pd
from app.sample_data import stat_vectors, comp_style_vectors, map_style_vectors, roles

In [10]:
BASE_DIR = os.path.abspath(os.path.dirname('../'))

SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))
hero_stats_path = f'{BASE_DIR}\\app\sample_data\hero_stats.csv'

meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

In [18]:
Hero=meta_data.tables['Hero']

hero_q = sa.select(
  Hero.c.HeroName
  , Hero.c.NewRole
)

hero_df = pd.read_sql(sql=hero_q, con=engine)
# hero_df.head()

In [20]:
hero_role_df = pd.DataFrame(columns=['HeroName', 'NewRole'])
hero_role_df = hero_role_df.astype(dtype= {"HeroName":"object" })
hero_role_df.HeroName = hero_df['HeroName']
hero_role_df.NewRole = hero_df[hero_role_df['HeroName'] == hero_df['HeroName']].NewRole

for col in roles:
  hero_role_df.loc[(hero_role_df.NewRole == col), col] = 1

hero_role_df.loc[(hero_role_df.HeroName == 'Varian'), 'Tank'] = 1
hero_role_df.loc[(hero_role_df.HeroName == 'Varian'), 'Melee Assassin'] = 1
hero_role_df.loc[(hero_role_df.HeroName == 'Kharazim'), 'Melee Assassin'] = 1
hero_role_df.loc[(hero_role_df.HeroName == 'Blaze'), 'Tank'] = 1

hero_role_df.fillna(0, inplace=True)
hero_role_df.head(100)

,HeroName,NewRole,Bruiser,Healer,Melee Assassin,Ranged Assassin,Support,Tank
0,Abathur,Support,0.0,0.0,0.0,0.0,1.0,0.0
1,Alarak,Melee Assassin,0.0,0.0,1.0,0.0,0.0,0.0
2,Alexstrasza,Healer,0.0,1.0,0.0,0.0,0.0,0.0
3,Ana,Healer,0.0,1.0,0.0,0.0,0.0,0.0
4,Anduin,Healer,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
85,Yrel,Bruiser,1.0,0.0,0.0,0.0,0.0,0.0
86,Zagara,Ranged Assassin,0.0,0.0,0.0,1.0,0.0,0.0
87,Zarya,Support,0.0,0.0,0.0,0.0,1.0,0.0
88,Zeratul,Melee Assassin,0.0,0.0,1.0,0.0,0.0,0.0


In [23]:
stats_df = pd.read_csv(hero_stats_path)
stats_df.set_index('HeroName', inplace=True)
stats_df.head()

,WaveClear,Engage,Peel,TeamSustain,SelfSustain,SoloLane,Fight,Macro,Artillery,Protect,Pick
HeroName,,,,,,,,,,,
Abathur,2.0,0.0,2.0,5.0,0.0,Maybe,4.0,8.0,1.0,7.0,2.0
Alarak,2.0,5.0,5.0,0.0,2.0,Maybe,5.0,3.0,2.0,4.0,9.0
Alexstrasza,4.0,1.0,2.0,7.0,5.0,Maybe,7.0,3.0,4.0,5.0,3.0
Ana,2.0,2.0,4.0,7.0,3.0,No,4.0,4.0,5.0,7.0,8.0
Anduin,3.0,2.0,4.0,6.0,5.0,No,4.0,3.0,3.0,7.0,7.0
